In [ ]:
# https://documentation.dataspace.copernicus.eu/notebook-samples/openeo/NDVI_Timeseries.html

In [1]:
from folium import GeoJson # Importing GeoJson class from folium library for visualizing geospatial data
import json # Importing json module for working with JSON data
import os # Importing os module for accessing operating system dependent functionality
import pandas as pd # Importing pandas library and aliasing it as pd for data analysis and manipulation
import matplotlib.pyplot as plt # Importing pyplot module from matplotlib library and aliasing it as plt for plotting
import scipy.signal # Importing signal module from scipy library for signal processing
import numpy as np # Importing numpy library and aliasing it as np for numerical computing
import geopandas as gpd # Importing geopandas library and aliasing it as gpd for working with geographic data
import openeo # Importing openeo library for working with the OpenEO API for Earth Observation data processing
from shapely.geometry import shape # Importing shape class from geometry module of shapely library for geometric operations
from shapely.geometry import Point # Importing Point class from geometry module of shapely library for representing a point in space

In [2]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [3]:
def read_json(filename: str) -> dict:
    with open(filename,encoding="utf-8") as input:
        field = json.load(input)
    return field

fields = read_json(r"C:\GIS_Course\EGM722\Project\Data\PZ_landfill_buff_1kmSqv4.geojson")


In [4]:
s2cube = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=["2021-08-01", "2021-10-31"],
    bands=["CH4"],
)

In [5]:
timeseries = s2cube.aggregate_spatial(geometries=fields, reducer="max")


In [7]:
job = timeseries.execute_batch(out_format="CSV", title="CH4 timeseries")

0:00:00 Job 'j-24041394f8ca42aa8af5121eda04240d': send 'start'


OpenEoApiError: [400] ConcurrentJobLimit: Job was not started because concurrent job limit (2) is reached. (ref: r-24041369dd1542c5aeb2174dc25b2715)

In [ ]:
job.get_results().download_file("CH4-results/CH4_timeseries.csv")

pd.read_csv("CH4-results/CH4_timeseries.csv", index_col=0)


In [ ]:
def plot_timeseries(filename, figsize=(15, 3)):
    df = pd.read_csv(filename, index_col=0)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    fig, ax = plt.subplots(figsize=figsize, dpi=90)
    df.groupby("feature_index")["max(band_0)"].plot(marker="o", ax=ax)
    ax.set_title(filename.split("/")[-1])
    ax.set_ylabel("CH4")
    ax.set_ylim(0, 5000)
    ax.legend(title="parcel id", loc='upper left', bbox_to_anchor=(1.02, 1), ncol=2)
    ax.xaxis.set_major_locator(plt.MaxNLocator(20))
    ax.grid(True)


In [ ]:
plot_timeseries("CH4-results/CH4_timeseries.csv")
